In [ ]:
import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

import math
import numpy as np
import pandas as pd
import scipy.signal as sig
import altair as alt

import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.load import graph_melt
from spinorama.load_parse import parse_graphs_speaker
from spinorama.compute_normalize import normalize
from spinorama.load_rewseq import parse_eq_iir_rews
from spinorama.graph import graph_spinorama, graph_freq
from spinorama.compute_normalize import normalize_mean, normalize_cea2034
from spinorama.compute_scores import speaker_pref_rating
from spinorama.compute_cea2034 import compute_cea2034, estimated_inroom_HV, spl2pressure, pressure2spl
from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build, peq_apply_measurements, peq_print

In [ ]:
# df_speaker = parse_graphs_speaker('Genelec', 'Genelec 8341A')
# df_speaker = parse_graphs_speaker('Focal', 'Focal Aria 906')
# df_speaker = parse_graphs_speaker('Elac', 'Elac Debut 2.0 B6.2')
df_speaker = parse_graphs_speaker('Adam', 'Adam S2V')

g_params = {'xmin': 20, 'xmax': 20000, 'ymin': -50, 'ymax': 10, 'width': 400, 'height': 250}
spin  = df_speaker['CEA2034_unmelted']
pir   = estimated_inroom_HV(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'])
score = speaker_pref_rating(graph_melt(spin), graph_melt(pir))

In [ ]:
my_fs = 48000
my_peq = parse_eq_iir_rews('./datas/eq/Adam S2V/iir.txt', my_fs)
ddf_horizontal = peq_apply_measurements(df_speaker['SPL Horizontal_unmelted'], my_peq)
ddf_vertical   = peq_apply_measurements(df_speaker['SPL Vertical_unmelted'], my_peq)

spin_filtered = normalize(graph_melt(compute_cea2034(ddf_horizontal, ddf_vertical)))
pir_filtered  = normalize(graph_melt(estimated_inroom_HV(ddf_horizontal, ddf_vertical)))
score_filtered = speaker_pref_rating(spin_filtered, pir_filtered)

In [ ]:
peq_print(my_peq)
print('         SPK FLT')
print('-----------------')
print('NBD  ON {0:0.2f} {1:0.2f}'.format(score['nbd_on_axis'], score_filtered['nbd_on_axis']))
print('NBD  LW {0:0.2f} {1:0.2f}'.format(score['nbd_listening_window'], score_filtered['nbd_listening_window']))
print('NBD PIR {0:0.2f} {1:0.2f}'.format(score['nbd_pred_in_room'], score_filtered['nbd_pred_in_room']))
print('SM  PIR {0:0.2f} {1:0.2f}'.format(score['sm_pred_in_room'], score_filtered['sm_pred_in_room']))
print('SM   SP {0:0.2f} {1:0.2f}'.format(score['sm_sound_power'], score_filtered['sm_sound_power']))
print('LFX       {0}   {1}'.format(score['lfx_hz'], score_filtered['lfx_hz']))
print('LFQ     {0} {1}'.format(score['lfq'], score_filtered['lfq']))
print('-----------------')
print('Score    {0}  {1}'.format(score['pref_score'], score_filtered['pref_score']))
print('-----------------')

In [ ]:
graph_spinorama(df_speaker['CEA2034'], g_params) | graph_spinorama(spin_filtered, g_params)

In [ ]:
def score_with_filter(splH, splV, peq):
    ddf_horizontal = peq_apply_measurements(splH, peq)
    ddf_vertical   = peq_apply_measurements(splV, peq)
    spin_filtered = normalize(graph_melt(compute_cea2034(ddf_horizontal, ddf_vertical)))
    pir_filtered  = normalize(graph_melt(estimated_inroom_HV(ddf_horizontal, ddf_vertical)))
    score_filtered = speaker_pref_rating(spin_filtered, pir_filtered, rounded=False)
    return -score_filtered['pref_score']

In [ ]:
my_fs = 48000
my_peq = [
    (0, Biquad(Biquad.HIGHPASS, 50, my_fs, 0.2, 1)),
    (1, Biquad(Biquad.PEAK, 715, my_fs, 8, -2.8)),
    (0, Biquad(Biquad.NOTCH, 1400, my_fs, 0.1, 0.1)),
]
score_with_filter(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'], my_peq)

In [ ]:
x0 = [700, 6, -2]

def objective(params):
    freq = params[0]
    Q    = params[1]
    Z    = params[2]
    the_peq = [(1, Biquad(Biquad.PEAK, freq, my_fs, Q, Z))]
    score = score_with_filter(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'], the_peq)
    print(params, score)
    return score

In [ ]:
objective(x0)

In [ ]:
from scipy.optimize import minimize, basinhopping

In [ ]:
bounds = [(20, 20000), (-10, 10), (0.1, 2)]
res = minimize(objective, x0, method='SLSQP', bounds=bounds)

res.x


In [ ]:

minimizer_kwargs = {"method": "COBYLA"}

def objective(params):
    freq = params[0]
    Q    = params[1]
    Z    = params[2]
    the_peq = [(1, Biquad(Biquad.PEAK, freq, my_fs, Q, Z))]
    score = score_with_filter(df_speaker['SPL Horizontal_unmelted'], df_speaker['SPL Vertical_unmelted'], the_peq)
    print(params, score)
    return score

class MyTakeStep(object):
   def __init__(self, stepsize=0.1):
       self.stepsize = stepsize
   def __call__(self, x):
       s = self.stepsize
       x[0] += np.random.uniform(-1000.*s, 1000.*s)
       x[1:] += np.random.uniform(-s, s, x[1:].shape)
       return x

class MyBounds(object):
    def __init__(self, xmax=[20000,10, 10], xmin=[20,-10, -10] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

mytakestep = MyTakeStep()
mybounds = MyBounds()

x0 = [700, 5, -2]
res = basinhopping(objective2, 
                   x0, 
                   minimizer_kwargs=minimizer_kwargs, 
                   take_step=mytakestep, 
                   accept_test=mybounds,
                   interval=5,
                   niter=2)